In [7]:
import sys

# put custom scripts to module path
sys.path.append('../../..')

In [8]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FormatStrFormatter

import numpy as np
import pandas as pd
import pathlib
import gym
import gym_fsw  # noqa: F401

from lcs import Perception
from lcs.metrics import population_metrics

from src.observation_wrappers import FSWObservationWrapper
from src.runner import run_experiments_alternating
from src.decorators import repeat, get_from_cache_or_run
from src.basic_rl import run_q_learning_alternating, run_r_learning_alternating, qlearning, rlearning
from src.visualization import diminishing_reward_colors, PLOT_DPI
from src.payoff_landscape import get_all_state_action, plot_payoff_landscape

from typing import List, Dict

plt.ioff()  # turn off interactive plotting
plt.style.use('../../../src/phd.mplstyle')

cache_dir = f'{pathlib.Path().absolute()}/cache'
plot_dir = f'{pathlib.Path().absolute()}/plots'


def common_metrics(agent, env):
    metrics = {}

    pop = agent.get_population()
    agent_name = agent.__class__.__name__

    if hasattr(agent, 'rho'):
        metrics['rho'] = agent.rho
        agent_name += "_v" + agent.cfg.rho_update_version
    else:
        metrics['rho'] = 0

    metrics['agent'] = agent_name
    metrics['reliable'] = len([cl for cl in pop if cl.is_reliable()])

    metrics.update(population_metrics(pop, env))

    return metrics


def average_experiment_runs(run_df: pd.DataFrame) -> pd.DataFrame:
    return run_df.groupby(['agent', 'trial', 'phase']).mean().reset_index(level='phase')

# Experiment 2 - Finite-State-World environment

In [9]:
learning_rate = 0.5
discount_factor = 0.95
epsilon = 0.1
zeta = 0.0001

def fsw_env_provider():
    return FSWObservationWrapper(gym.make(f'fsw-10-v0'))

# Set ACS2/AACS2 configuration parameter dictionary
basic_cfg = {
    'perception_bits': 1,
    'possible_actions': 2,
    'do_ga': False,
    'beta': learning_rate,
    'epsilon': epsilon,
    'gamma': discount_factor,
    'zeta': zeta,
    'user_metrics_collector_fcn': common_metrics,
    'biased_exploration_prob': 0,
    'metrics_trial_freq': 1
}

NUM_EXPERIMENTS = 10
trials = 10_000


@get_from_cache_or_run(cache_path=f'{cache_dir}/fsw/acs2.dill')
@repeat(num_times=NUM_EXPERIMENTS)
def run_acs2_in_fsw():
    return run_experiments_alternating(fsw_env_provider, trials, basic_cfg)


@get_from_cache_or_run(cache_path=f'{cache_dir}/fsw/qlearning.dill')
def run_qlearning_in_fsw():
    fsw_env = fsw_env_provider()
    init_Q = np.zeros((fsw_env.observation_space.n, fsw_env.action_space.n))
    return run_q_learning_alternating(NUM_EXPERIMENTS, trials, fsw_env, epsilon, learning_rate, discount_factor,
                                      init_Q, perception_to_state_mapper=lambda p: int(p[0]))


@get_from_cache_or_run(cache_path=f'{cache_dir}/fsw/rlearning.dill')
def run_rlearning_in_fsw():
    fsw_env = fsw_env_provider()
    init_R = np.zeros((fsw_env.observation_space.n, fsw_env.action_space.n))
    return run_r_learning_alternating(NUM_EXPERIMENTS, trials, fsw_env, epsilon, learning_rate, zeta, init_R,
                                      perception_to_state_mapper=lambda p: int(p[0]))


# run computations
acs2_runs_details = run_acs2_in_fsw()
q_learning_metrics = run_qlearning_in_fsw()
r_learning_metrics = run_rlearning_in_fsw()

# average runs and create aggregated metrics data frame
acs2_metrics = [m_df for _, _, _, m_df in acs2_runs_details]

agg_df = pd.concat([
    average_experiment_runs(pd.concat(acs2_metrics)),
    average_experiment_runs(pd.DataFrame(q_learning_metrics)),
    average_experiment_runs(pd.DataFrame(r_learning_metrics))]
)

100%|██████████| 10/10 [00:20<00:00,  2.03s/it]


In [10]:
def plot_pop_and_rho(df, plot_filename=None):
    colors = diminishing_reward_colors()

    expl_df = df[df['phase'] == 'exploit']

    xmax = trials/2

    fig, axs = plt.subplots(2, 1, figsize=(18, 16))

    # Steps in trial plot
    for alg in ['ACS2', 'AACS2_v1', 'AACS2_v2', 'Q-Learning', 'R-Learning']:
        alg_df = expl_df.loc[alg]
        idx = pd.Index(name='exploit trial', data=np.arange(1, len(alg_df) + 1))
        alg_df.set_index(idx, inplace=True)

        alg_df['steps_in_trial'].rolling(window=10).mean().plot(ax=axs[0], label=alg, linewidth=2, color=colors[alg])

    axs[0].set_xlabel("Exploit trial")
    axs[0].set_xlim(1, 500)
    axs[0].xaxis.set_major_locator(MultipleLocator(50))
    axs[0].xaxis.set_minor_locator(MultipleLocator(10))
    axs[0].xaxis.set_major_formatter(FormatStrFormatter('%1.0f'))
    axs[0].xaxis.set_tick_params(which='major', size=10, width=2, direction='in')
    axs[0].xaxis.set_tick_params(which='minor', size=5, width=1, direction='in')

    axs[0].set_ylabel("Number of steps")
    axs[0].yaxis.set_major_locator(MultipleLocator(1))
    axs[0].yaxis.set_tick_params(which='major', size=10, width=2, direction='in')
    axs[0].yaxis.set_tick_params(which='minor', size=5, width=1, direction='in')

    axs[0].set_title('Steps in trial')
    axs[0].legend(loc='upper right', frameon=False)

    # Rho plot
    for alg in ['AACS2_v1', 'AACS2_v2', 'R-Learning']:
        alg_df = expl_df.loc[alg]
        idx = pd.Index(name='exploit trial', data=np.arange(1, len(alg_df) + 1))
        alg_df.set_index(idx, inplace=True)

        alg_df['rho'].plot(ax=axs[1], label=alg, linewidth=2, color=colors[alg])

    axs[1].set_xlim(0, xmax)
    axs[1].set_xlabel("Exploit trial")
    axs[1].xaxis.set_major_locator(MultipleLocator(500))
    axs[1].xaxis.set_minor_locator(MultipleLocator(100))
    axs[1].xaxis.set_major_formatter(FormatStrFormatter('%1.0f'))
    axs[1].xaxis.set_tick_params(which='major', size=10, width=2, direction='in')
    axs[1].xaxis.set_tick_params(which='minor', size=5, width=1, direction='in')

    axs[1].set_ylabel(r"$\mathregular{\rho}$")
    axs[1].yaxis.set_major_locator(MultipleLocator(2))
    axs[1].yaxis.set_minor_locator(MultipleLocator(1))
    axs[1].yaxis.set_tick_params(which='major', size=10, width=2, direction='in')
    axs[1].yaxis.set_tick_params(which='minor', size=5, width=1, direction='in')
    axs[1].set_ylim(0, 11)

    axs[1].set_title(r'Estimated average $\mathregular{\rho}$')

    if plot_filename:
            fig.savefig(plot_filename, dpi=PLOT_DPI, bbox_inches='tight')

plot_pop_and_rho(agg_df, plot_filename=f'{plot_dir}/fsw-performance.png')

:::{figure-md} 51-fsw-fig
:class: full-width
<img src="plots/fsw-performance.png">

Performance in FSW environment
:::

## Payoff landscape

In [11]:
fsw_env = fsw_env_provider()
state_action = get_all_state_action(fsw_env.state_action())


def calculate_state_action_payoffs(state_actions: List, pop_acs2, pop_aacs2v1, pop_aacs2v2, Q, R) -> Dict:
    payoffs = {}

    for sa in state_actions:
        p = Perception((sa.state,))

        # ACS2
        acs2_match_set = pop_acs2.form_match_set(p)
        acs2_action_set = acs2_match_set.form_action_set(sa.action)

        # AACS2_v1
        aacs2v1_match_set = pop_aacs2v1.form_match_set(p)
        aacs2v1_action_set = aacs2v1_match_set.form_action_set(sa.action)

        # AACS2_v2
        aacs2v2_match_set = pop_aacs2v2.form_match_set(p)
        aacs2v2_action_set = aacs2v2_match_set.form_action_set(sa.action)

        # Check if all states are covered
        for alg, action_set in zip(['ACS2', 'AACS2_v1', 'AACS2_v2'],
                                   [acs2_action_set, aacs2v1_action_set,
                                    aacs2v2_action_set]):
            if len(action_set) == 0:
                print(f"WARN: No {alg} classifiers for perception: {p}, action: {sa.action}")

        payoffs[sa] = {
            'ACS2': np.mean(list(map(lambda cl: cl.r, acs2_action_set))),
            'AACS2_v1': np.mean(list(map(lambda cl: cl.r, aacs2v1_action_set))),
            'AACS2_v2': np.mean(list(map(lambda cl: cl.r, aacs2v2_action_set))),
            'Q-Learning': Q[int(sa.state), sa.action],
            'R-Learning': R[int(sa.state), sa.action]
        }

    return payoffs

# Take first of each algorithm population pass for presenting payoff landscape
pop_acs2, pop_aacs2v1, pop_aacs2v2, _ = acs2_runs_details[0]

@get_from_cache_or_run(cache_path=f'{cache_dir}/fsw/qlearning-single.dill')
def run_single_qlearning():
    Q_init = np.zeros((fsw_env.observation_space.n, fsw_env.action_space.n))
    Q, _ = qlearning(fsw_env, trials, Q_init, epsilon, learning_rate, discount_factor, perception_to_state_mapper=lambda p: int(p[0]))
    return Q

@get_from_cache_or_run(cache_path=f'{cache_dir}/fsw/rlearning-single.dill')
def run_single_rlearning():
    R_init = np.zeros((fsw_env.observation_space.n, fsw_env.action_space.n))
    R, rho, _ = rlearning(fsw_env, trials, R_init, epsilon, learning_rate, zeta, perception_to_state_mapper=lambda p: int(p[0]))
    return R, rho

Q = run_single_qlearning()
R, rho = run_single_rlearning()

payoffs = calculate_state_action_payoffs(state_action, pop_acs2, pop_aacs2v1, pop_aacs2v2, Q, R)

R-Learning: 100%|██████████| 10000/10000 [00:01<00:00, 5292.54it/s]


In [12]:
plot_payoff_landscape(payoffs, rho=rho, rho_text_location={'x': 30, 'y': 60}, plot_filename=f'{plot_dir}/fsw-payoff-landscape.png')

:::{figure-md} 51-fsw-payoff-landscape-fig
:class: full-width
<img src="plots/fsw-payoff-landscape.png">

FSW payoff landscape
:::